### 1. 필요한 라이브러리 설치

In [82]:
pip install openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
pip install soundfile


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.0 MB 991.0 kB/s eta 0:00:01
   -------- ------------------------------- 0.2/1.0 MB 2.5 MB/s eta 0:00:01
   --------------- ------------------------ 0.4/1.0 MB 3.0 MB/s eta 0:00:01
   ------------------------------------- -- 1.0/1.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 5.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [85]:
pip install pydub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
pip install requests

   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 1.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/126.3 kB ? eta -:--:--
   ---------------------------------------- 126.3/126.3 kB 7.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [95]:
pip install numpy

     ---------------------------------------- 0.0/59.7 kB ? eta -:--:--
     ------------- -------------------------- 20.5/59.7 kB ? eta -:--:--
     -------------------------------------- 59.7/59.7 kB 785.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.6 MB 3.6 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/12.6 MB 22.0 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/12.6 MB 22.0 MB/s eta 0:00:01
   ---------- ----------------------------- 3.1/12.6 MB 20.1 MB/s eta 0:00:01
   ---------- ----------------------------- 3.1/12.6 MB 20.1 MB/s eta 0:00:01
   ------------------------- -------------- 8.0/12.6 MB 32.0 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.6 MB 46.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.6 MB 46.9 MB/s eta 0:00:01
   --------------------------------- ------ 10.5/12.6 MB 46.9 MB/s eta 0:00:01
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import queue
import threading
import sounddevice as sd
import soundfile as sf
import time
from pydub import AudioSegment
import requests
import json
import base64
import urllib3
import os
from openai import OpenAI

# OpenAI API 키 설정
client = OpenAI(api_key='sk-VsBg3163eZFviFyybH3yorlp8TE5WlPQOeh_qUjLReT3BlbkFJ-ZZNFazlMsQ2_JF28W1TIgfbG9_B_a7nnASC8VBnYA')


### 2. 전역 변수 설정

In [89]:
q = queue.Queue()
recording = False

### 3. 실시간 녹음 함수 및 콜백 함수

In [90]:
def complicated_record():
    with sf.SoundFile("temp.wav", mode='w', samplerate=16000, subtype='PCM_16', channels=1) as file:
        with sd.InputStream(samplerate=16000, dtype='int16', channels=1, callback=complicated_save):
            while recording:
                file.write(q.get())

def complicated_save(indata, frames, time, status):
    q.put(indata.copy())


### 4. 녹음 시작 및 종료

In [91]:
def start_recording():
    global recording
    recording = True
    recorder = threading.Thread(target=complicated_record)
    print('Start recording')
    recorder.start()
    return recorder

def stop_recording(recorder):
    global recording
    recording = False
    recorder.join()
    print('Stop recording')


### 5. STT 변환 및 파일 삭제

In [92]:
def split_audio_and_send(file_path, segment_length=20000):
    audio = AudioSegment.from_wav(file_path)
    duration = len(audio)
    full_text = ""
    
    for i in range(0, duration, segment_length):
        segment = audio[i:i + segment_length]
        segment_path = f"segment_{i//1000}.wav"
        segment.export(segment_path, format="wav")
        text = send_to_stt(segment_path)
        full_text += text + " "
        
        if os.path.exists(segment_path):
            os.remove(segment_path)

    return full_text.strip()

def send_to_stt(segment_path):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
    accessKey = "your-etri-access-key"
    languageCode = "korean"

    with open(segment_path, "rb") as file:
        audioContents = base64.b64encode(file.read()).decode("utf8")
    
    requestJson = {
        "argument": {
            "language_code": languageCode,
            "audio": audioContents
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": accessKey},
        body=json.dumps(requestJson)
    )
    
    response_data = json.loads(response.data.decode('utf-8'))
    return response_data.get("return_object", {}).get("recognized", "")


### 6. GPT로 대화 방향성 조언 받기

In [93]:
def get_gpt_advice(conversation_text):
    # GPT 모델에게 보내는 메시지 형식
    messages = [
        {"role": "system", "content": "You are a helpful assistant providing conversation guidance for welfare services."},
        {"role": "user", "content": f"Here is the conversation so far: {conversation_text}. What should be the next step to provide the best advice and support?"}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Chat-based GPT 모델
        messages=messages,
        max_tokens=300,
        temperature=0.7,
        top_p=1.0,
        n=1
    )
    
    # GPT의 응답 반환
    return response.choices[0].message.content.strip()


### 7. 메인 함수

In [94]:
# 60초 동안 녹음
recorder_thread = start_recording()
time.sleep(60)
stop_recording(recorder_thread)

# STT 변환
full_text = split_audio_and_send("temp.wav")
print("Full Recognized Text:", full_text)

# GPT에게 조언 요청
advice = get_gpt_advice(full_text)
print("GPT Advice:", advice)

# 원본 오디오 파일 삭제
if os.path.exists("temp.wav"):
    os.remove("temp.wav")
    print("Original audio file deleted.")


Exception in thread Thread-104 (complicated_record):
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\USER\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_2600\3078862851.py", line 3, in complicated_record
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sounddevice.py", line 1429, in __init__
    _StreamBase.__init__(self, kind='input', wrap_callback='array',
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\sounddevice.py", line 777, in __init__
    import numpy
ModuleNotFoundError: No module named 'numpy'


Start recording
Stop recording
Full Recognized Text: 
GPT Advice: It looks like there may have been a mistake in your message. Could you please provide more details or context so I can offer you the best guidance possible?
Original audio file deleted.


sk-VsBg3163eZFviFyybH3yorlp8TE5WlPQOeh_qUjLReT3BlbkFJ-ZZNFazlMsQ2_JF28W1TIgfbG9_B_a7nnASC8VBnYA

In [66]:
from openai import OpenAI

# OpenAI API 키 설정
client = OpenAI(api_key='sk-VsBg3163eZFviFyybH3yorlp8TE5WlPQOeh_qUjLReT3BlbkFJ-ZZNFazlMsQ2_JF28W1TIgfbG9_B_a7nnASC8VBnYA')

In [68]:

def generate_conversation_guideline(user_input, target_audience):
    """
    사용자 입력과 대상에 맞춘 대화 가이드라인을 GPT를 통해 생성하는 함수

    Args:
        user_input (str): 사용자로부터 입력받은 복지 관련 질문 또는 주제
        target_audience (str): 복지 대상 (예: 노인, 장애인 등)
    
    Returns:
        str: GPT로 생성된 대화 가이드라인
    """
    
    # GPT-3.5 모델에게 보낼 프롬프트 설정
    messages = [
        {"role": "system", "content": "You are an expert in welfare services."},
        {"role": "user", "content": f"Create a conversation guideline for {target_audience} welfare services. The topic is '{user_input}'. Provide a structured dialogue guide with helpful questions and steps to follow."}
    ]
    
    # OpenAI API를 통해 응답 생성
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # Chat-based GPT 모델 설정
        messages=messages,
        max_tokens=500,  # 생성할 응답의 최대 토큰 수
        temperature=0.7,  # 창의성 설정 (0.0 ~ 1.0)
        top_p=1.0,  # 확률 분포를 기반으로 한 선택의 다양성
        n=1  # 결과 개수
    )
    
    # 결과 반환 (content 속성을 직접 사용)
    return response.choices[0].message.content.strip()

# 사용자 입력 예시
user_input = "elderly care services"
target_audience = "elderly"

# 대화 가이드라인 생성
guideline = generate_conversation_guideline(user_input, target_audience)

# 결과 출력
print("Generated Conversation Guideline:")
print(guideline)

Generated Conversation Guideline:
Conversation Guideline for Elderly Care Services:

Step 1: Introduction and Establishing Rapport
1. Start by introducing yourself and your role in the welfare services.
2. Greet the elderly individual warmly and ask how they are feeling today.
3. Establish rapport by showing genuine interest and empathy towards their well-being.

Step 2: Understanding the Elderly Individual's Needs
4. Ask the elderly individual about their current living situation and any challenges they may be facing.
5. Inquire about their daily activities and routines to understand their lifestyle better.
6. Explore any health issues or medical conditions they may have and how it impacts their daily life.
7. Discuss their social support system and if they have any caregivers or family members assisting them.

Step 3: Discussing Available Elderly Care Services
8. Explain the different types of elderly care services available such as home care, assisted living, or nursing homes.
9. Pr

In [97]:
def generate_real_time_conversation_response(user_input, conversation_context):
    """
    실시간 대화를 위한 GPT 대화 응답 생성 함수

    Args:
        user_input (str): 상대방이 말한 대화 내용
        conversation_context (str): 대화의 전체 흐름과 맥락 (이전 대화 내용)
    
    Returns:
        str: GPT로 생성된 실시간 응답
    """
    
    # GPT 모델에 보낼 프롬프트 설정
    messages = [
        {"role": "system", "content": "당신은 사회복지 서비스 전문가입니다. 상대방의 말에 맞춰 적절하고 따뜻한 응답을 생성합니다."},
        {
            "role": "user", 
            "content": f"""
            이전 대화 내용: '{conversation_context}'
            상대방이 말한 내용: '{user_input}'
            
            상대방의 말에 적절하게 응답해 주세요. 응답은 상대방의 감정 상태를 고려하고, 필요한 경우 조언이나 추가 질문을 포함하여 대화가 자연스럽게 이어질 수 있도록 해 주세요.
            """
        }
    ]
    
    # OpenAI API 호출
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # 모델 설정
        messages=messages,
        max_tokens=500,  # 생성할 응답의 최대 토큰 수
        temperature=0.7,  # 창의성 설정
        top_p=1.0,  # 확률 분포 기반 선택
        n=1  # 생성할 결과 개수
    )
    
    # 생성된 응답 반환
    return response.choices[0].message.content.strip()

# 대화의 맥락 예시
conversation_context = "안녕하세요, 오늘 기분이 어떠신가요? 조금 피곤하지만 괜찮아요."

# 상대방의 실시간 발언
user_input = "무릎이 계속 아프면 병원에 가보시는 게 좋을 것 같아요. 혹시 최근에 병원에 가신 적 있나요?"

# 실시간 대화 응답 생성
response = generate_real_time_conversation_response(user_input, conversation_context)

# 결과 출력
print("Generated Real-Time Response:")
print(response)


Generated Real-Time Response:
무릎이 계속 아프시다니, 정말 힘드시겠네요. 병원에 가보시는 것이 좋은 선택일 수 있어요. 최근에 병원에 가실 때의 상황이 궁금하네요. 혹시 무엇 때문에 무릎이 아프신 건가요? 함께 대화해보면 무릎 통증을 완화할 수 있는 방법에 대해 이야기해볼 수도 있겠죠. 함께 건강을 챙기는 것이 중요하니까요.
